# Inputs

In [1]:
splitPartCount = 5
splitSeed = 27
hoursPerWindow = 1

`functionize-notebook` has modified this notebook during execution. The following variables have been injected:

- hoursPerWindow: 1


# Preproccess

## read data

In [3]:
from utils.class_patient import Patients


patients = Patients.loadPatients()
len(patients)

1213

## remove missing

In [4]:
# fill measures whose null represent false value

from constants import NULLABLE_MEASURES


nullableMeasures = NULLABLE_MEASURES

for measureName in nullableMeasures:
    patients.fillMissingMeasureValue(measureName, 0)

In [5]:
# remove measures with less than 80% of data

measures = patients.getMeasures()

for measure, count in measures.items():
    if count < len(patients) * 80 / 100:
        patients.removeMeasures([measure])
        print(measure, count)

pco2 917
ph 954
po2 917
albumin 406
hba1c 326
lymphocyte 446
height 415
urine-ketone 294
crp 19


In [6]:
# remove patients with less than 80% of data

patients.removePatientByMissingFeatures()
len(patients)

1209

## split patients

In [7]:
splitedPatients = patients.split(splitPartCount, splitSeed)

len(splitedPatients[0])

242

In [8]:
from pandas import Timedelta


splitedPatients[0].getMeasuresBetween(Timedelta(-6), Timedelta(1))

,subject_id,hadm_id,stay_id,akd,ag,age,bg,bicarbonate,bun,calcium,...,race,rr,saps2,sbp,scr,sofa,use_NaHCO3,uti,wbc,weight
0,19054290,20046699,30643955,False,NaN,65,NaN,NaN,NaN,NaN,...,WHITE,NaN,34,NaN,NaN,3,0.0,0,NaN,74.50
0,14866589,20066152,33060916,False,NaN,37,NaN,NaN,NaN,NaN,...,WHITE,NaN,22,NaN,NaN,2,0.0,0,NaN,NaN
0,14849360,20152551,32817197,False,NaN,59,NaN,NaN,NaN,NaN,...,UNABLE TO OBTAIN,NaN,31,NaN,NaN,7,0.0,0,NaN,NaN
0,16171124,20167052,38239449,False,NaN,37,NaN,NaN,NaN,NaN,...,HISPANIC/LATINO - DOMINICAN,NaN,15,NaN,NaN,2,0.0,0,NaN,NaN
0,16815664,20240670,37751533,False,NaN,42,NaN,NaN,NaN,NaN,...,BLACK/AFRICAN AMERICAN,NaN,27,NaN,NaN,2,NaN,0,NaN,43.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
0,15517352,29801212,32814591,False,NaN,20,NaN,NaN,NaN,NaN,...,WHITE,NaN,30,NaN,NaN,4,NaN,0,NaN,NaN
0,10383045,29899941,39755932,False,NaN,58,NaN,NaN,NaN,NaN,...,WHITE,NaN,23,NaN,NaN,1,0.0,0,NaN,59.40
0,15679298,29905973,39834726,False,NaN,50,NaN,NaN,NaN,NaN,...,WHITE,NaN,32,NaN,NaN,3,0.0,0,NaN,47.40
0,10912213,29911812,34040470,False,NaN,47,NaN,NaN,NaN,NaN,...,WHITE,NaN,23,NaN,NaN,2,0.0,0,NaN,89.05


In [9]:
splitedPatients = patients.split(splitPartCount, splitSeed)


def trainTest():
    for i in range(splitedPatients.__len__()):
        testPatients = splitedPatients[i]

        trainPatientsList = splitedPatients[:i] + splitedPatients[i + 1 :]
        trainPatients = Patients(patients=[])
        for trainPatientsElem in trainPatientsList:
            trainPatients += trainPatientsElem

        yield trainPatients, testPatients


def trainValTest():
    for i in range(splitedPatients.__len__()):
        testPatients = splitedPatients[i]

        trainPatientsList = splitedPatients[:i] + splitedPatients[i + 1 :]
        trainPatients = Patients(patients=[])
        for trainPatientsElem in trainPatientsList:
            trainPatients += trainPatientsElem

        *trainPatients, valPatients = trainPatients.split(5, 27)
        tmpPatients = Patients(patients=[])
        for trainPatientsElem in trainPatients:
            tmpPatients += trainPatientsElem
        trainPatients = tmpPatients

        yield trainPatients, valPatients, testPatients

In [10]:
for trainPatients, testPatients in trainTest():
    print(len(trainPatients.patientList), len(testPatients.patientList))

967 242
967 242
967 242
967 242
968 241


In [11]:
from constants import CATEGORICAL_MEASURES, TEMP_PATH
from utils.prepare_data import patientsToNumpy


dataset = []
for i, (trainPatients, testPatients) in enumerate(trainTest()):
    # trainPatients.fillMissingMeasureValue(CATEGORICAL_MEASURES, 0)
    npTrainX, categoryEncoder, numericEncoder, oulier, columns = patientsToNumpy(
        trainPatients, hoursPerWindow, CATEGORICAL_MEASURES
    )

    # testPatients.fillMissingMeasureValue(CATEGORICAL_MEASURES, 0)
    npTestX, *_ = patientsToNumpy(
        testPatients,
        hoursPerWindow,
        CATEGORICAL_MEASURES,
        columns,
        categoryEncoder,
        numericEncoder,
        oulier,
    )
    trainY = [p.akdPositive for p in trainPatients]
    testY = [p.akdPositive for p in testPatients]

    dataset.append((npTrainX, trainY, npTestX, testY))

    print(f"Dataset {i} created")
    pass

Dataset 0 created


Dataset 1 created


Dataset 2 created


Dataset 3 created


Dataset 4 created


In [12]:
import pickle


(TEMP_PATH / f"train-test-set-{splitPartCount}-{splitSeed}-{hoursPerWindow}.pkl").write_bytes(
    pickle.dumps(dataset)
)

41441534

In [13]:
from constants import CATEGORICAL_MEASURES, TEMP_PATH
from utils.prepare_data import patientsToNumpy


datavalset = []
for i, (trainPatients, valPatients, testPatients) in enumerate(trainValTest()):
    # trainPatients.fillMissingMeasureValue(CATEGORICAL_MEASURES, 0)
    npTrainX, categoryEncoder, numericEncoder, oulier, columns = patientsToNumpy(
        trainPatients, hoursPerWindow, CATEGORICAL_MEASURES
    )

    # testPatients.fillMissingMeasureValue(CATEGORICAL_MEASURES, 0)
    npTestX, *_ = patientsToNumpy(
        testPatients,
        hoursPerWindow,
        CATEGORICAL_MEASURES,
        columns,
        categoryEncoder,
        numericEncoder,
        oulier,
    )

    npValX, *_ = patientsToNumpy(
        valPatients,
        hoursPerWindow,
        CATEGORICAL_MEASURES,
        columns,
        categoryEncoder,
        numericEncoder,
        oulier,
    )

    trainY = [p.akdPositive for p in trainPatients]
    testY = [p.akdPositive for p in testPatients]
    valY = [p.akdPositive for p in valPatients]

    datavalset.append((npTrainX, trainY, npValX, valY, npTestX, testY))

    print(f"Dataset {i} created")
    pass

Dataset 0 created


Dataset 1 created


Dataset 2 created


Dataset 3 created


Dataset 4 created


In [14]:
import pickle

(TEMP_PATH / f"train-val-test-set-{splitPartCount}-{splitSeed}-{hoursPerWindow}.pkl").write_bytes(
    pickle.dumps(datavalset)
)

40861464

# LSTM

In [15]:
import pickle
from constants import TEMP_PATH

# load if not exist dataset
if "dataset" not in locals():
    dataset = pickle.loads(
        (TEMP_PATH / f"dataset-{splitPartCount}-{splitSeed}-{hoursPerWindow}.pkl").read_bytes()
    )
   

In [16]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
from tensorflow.keras.optimizers import Adam

def createModel(timeSteps, features):
    model = Sequential()
    model.add(LSTM(64, input_shape=(timeSteps, features), return_sequences=True))
    model.add(Dropout(0.5))
    model.add(LSTM(32))
    model.add(Dropout(0.5))
    model.add(Dense(16, activation="relu"))
    model.add(Dense(1, activation="sigmoid"))

    optimizer = Adam(learning_rate=0.001)
    model.compile(optimizer=optimizer, loss="binary_crossentropy", metrics=["AUC", "accuracy", "precision", "recall"])
    return model

2024-06-07 11:55:46.220754: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-06-07 11:55:46.238606: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


2024-06-07 11:55:46.508142: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [17]:
# crossfold
import numpy as np
from tensorflow.keras.callbacks import EarlyStopping


loses = []
aucs = []
accuracies = []
precisions = []
recals = []
for i, (npTrainX, trainY, npTestX, testY) in enumerate(dataset):
    npTrainX = np.nan_to_num(npTrainX, nan=0)
    npTestX = np.nan_to_num(npTestX, nan=0)

    model = createModel(npTrainX.shape[1], npTrainX.shape[2])

    neg, pos = np.bincount(trainY)
    weight0 = (1 / neg) * (len(trainY)) / 2.0
    weight1 = (1 / pos) * (len(trainY)) / 2.0
    weight = {0: weight0, 1: weight1}

    early_stopping = EarlyStopping(
        monitor="val_loss", patience=10, restore_best_weights=True
    )

    trainY = np.array(trainY)
    model.fit(
        npTrainX,
        trainY,
        epochs=50,
        batch_size=32,
        validation_split=0.2,
        class_weight=weight,
        callbacks=[early_stopping],
    )

    testY = np.array(testY)
    loss, auc, accuracy, precison, recal = model.evaluate(npTestX, testY)
    loses.append(loss)
    aucs.append(auc)
    accuracies.append(accuracy)
    precisions.append(precison)
    recals.append(recal)

    pass

print("Loses:", loses, np.mean(loses), np.std(loses))
print("AUCs:", aucs, np.mean(aucs), np.std(aucs))
print("Accuracies:", accuracies, np.mean(accuracies), np.std(accuracies))
print("Precisions:", precisions, np.mean(precisions), np.std(precisions))
print("Recals:", recals, np.mean(recals), np.std(recals))

2024-06-07 11:55:46.711084: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:998] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355


Epoch 1/50


2024-06-07 11:55:46.725145: W tensorflow/core/common_runtime/gpu/gpu_device.cc:2251] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...
/home/tu/codepy/hust.year2023.PredictingRiskDiabeticKetoacidosis-associatedKidneyInjury/.venv/lib/python3.12/site-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


 1/25 ━━━━━━━━━━━━━━━━━━━━ 32s 1s/step - AUC: 0.2146 - accuracy: 0.2812 - loss: 0.7341 - precision: 0.1765 - recall: 0.2500

10/25 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - AUC: 0.4940 - accuracy: 0.5082 - loss: 0.6857 - precision: 0.3491 - recall: 0.4456

...

15/25 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - AUC: 0.9681 - accuracy: 0.9221 - loss: 0.2268 - precision: 0.8923 - recall: 0.8972 

25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - AUC: 0.9662 - accuracy: 0.9190 - loss: 0.2310 - precision: 0.8897 - recall: 0.8969 - val_AUC: 0.7327 - val_accuracy: 0.7320 - val_loss: 0.8786 - val_precision: 0.6533 - val_recall: 0.6533


1/8 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - AUC: 0.7812 - accuracy: 0.7812 - loss: 0.5622 - precision: 0.7647 - recall: 0.8125

8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - AUC: 0.7752 - accuracy: 0.7381 - loss: 0.5720 - precision: 0.6579 - recall: 0.7401 


Loses: [0.5899850726127625, 0.5520228743553162, 0.5021911263465881, 0.5767034292221069, 0.5864744186401367] 0.5614753842353821 0.032476516386832285
AUCs: [0.7685284614562988, 0.7965986728668213, 0.8368062973022461, 0.7702268362045288, 0.7665824890136719] 0.7877485513687134 0.026867867949518946
Accuracies: [0.6818181872367859, 0.7438016533851624, 0.797520637512207, 0.7190082669258118, 0.7178423404693604] 0.7319982171058654 0.03826139498396454
Precisions: [0.5762711763381958, 0.6542056202888489, 0.7254902124404907, 0.6320754885673523, 0.6261682510375977] 0.642842149734497 0.048549902412810465
Recals: [0.7157894968986511, 0.7368420958518982, 0.7789473533630371, 0.6979166865348816, 0.7052631378173828] 0.7269517540931701 0.0291183392839851


## Static validate 

In [18]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
from tensorflow.keras.optimizers import Adam


def createModel(timeSteps, features):
    model = Sequential()
    model.add(LSTM(64, input_shape=(timeSteps, features), return_sequences=True))
    model.add(Dropout(0.5))
    model.add(LSTM(32))
    model.add(Dropout(0.5))
    model.add(Dense(16, activation="relu"))
    model.add(Dense(1, activation="sigmoid"))

    optimizer = Adam(learning_rate=0.001)
    model.compile(
        optimizer=optimizer,
        loss="binary_crossentropy",
        metrics=["AUC", "accuracy", "precision", "recall"],
    )
    return model

In [19]:
# crossfold
import numpy as np
from tensorflow.keras.callbacks import EarlyStopping

if "datavalset" not in locals():
    datavalset = pickle.loads(
        (TEMP_PATH / f"train-val-test-set-{splitPartCount}-{splitSeed}-{hoursPerWindow}.pkl").read_bytes()
    )

loses = []
aucs = []
accuracies = []
precisions = []
recals = []
for i, (npTrainX, trainY, npValX, valY, npTestX, testY) in enumerate(datavalset):
    npTrainX = np.nan_to_num(npTrainX, nan=0)
    npValX = np.nan_to_num(npValX, nan=0)
    npTestX = np.nan_to_num(npTestX, nan=0)

    model = createModel(npTrainX.shape[1], npTrainX.shape[2])

    neg, pos = np.bincount(trainY)
    weight0 = (1 / neg) * (len(trainY)) / 2.0
    weight1 = (1 / pos) * (len(trainY)) / 2.0
    weight = {0: weight0, 1: weight1}

    early_stopping = EarlyStopping(
        monitor="val_loss", patience=10, restore_best_weights=True
    )

    trainY = np.array(trainY)
    valY = np.array(valY)
    testY = np.array(testY)

    model.fit(
        npTrainX,
        trainY,
        epochs=50,
        batch_size=32,
        validation_data=(npValX, valY),
        class_weight=weight,
        callbacks=[early_stopping],
    )

    loss, auc, accuracy, precison, recal = model.evaluate(npTestX, testY)
    loses.append(loss)
    aucs.append(auc)
    accuracies.append(accuracy)
    precisions.append(precison)
    recals.append(recal)

    pass

print("Loses:", loses, np.mean(loses), np.std(loses))
print("AUCs:", aucs, np.mean(aucs), np.std(aucs))
print("Accuracies:", accuracies, np.mean(accuracies), np.std(accuracies))
print("Precisions:", precisions, np.mean(precisions), np.std(precisions))
print("Recals:", recals, np.mean(recals), np.std(recals))

Epoch 1/50


 1/25 ━━━━━━━━━━━━━━━━━━━━ 27s 1s/step - AUC: 0.3571 - accuracy: 0.3750 - loss: 0.7571 - precision: 0.0000e+00 - recall: 0.0000e+00

11/25 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - AUC: 0.5054 - accuracy: 0.5420 - loss: 0.7127 - precision: 0.2688 - recall: 0.0701        

22/25 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - AUC: 0.5766 - accuracy: 0.5752 - loss: 0.6989 - precision: 0.4167 - recall: 0.1418

25/25 ━━━━━━━━━━━━━━━━━━━━ 2s 15ms/step - AUC: 0.5969 - accuracy: 0.5878 - loss: 0.6920 - precision: 0.4490 - recall: 0.1708 - val_AUC: 0.7221 - val_accuracy: 0.6684 - val_loss: 0.5990 - val_precision: 0.5698 - val_recall: 0.6447


...

14/25 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - AUC: 0.9655 - accuracy: 0.9253 - loss: 0.2162 - precision: 0.8844 - recall: 0.9471 

25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - AUC: 0.9636 - accuracy: 0.9195 - loss: 0.2285 - precision: 0.8707 - recall: 0.9454 - val_AUC: 0.7568 - val_accuracy: 0.6839 - val_loss: 0.8270 - val_precision: 0.5728 - val_recall: 0.7763


1/8 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - AUC: 0.7812 - accuracy: 0.7500 - loss: 0.5795 - precision: 0.7000 - recall: 0.8750

8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - AUC: 0.7833 - accuracy: 0.7084 - loss: 0.5974 - precision: 0.6030 - recall: 0.8310 


Loses: [0.6005104184150696, 0.5677282214164734, 0.5037206411361694, 0.5897627472877502, 0.6133068203926086] 0.5750057697296143 0.03865167070423516
AUCs: [0.7453633546829224, 0.7805585265159607, 0.8494449853897095, 0.75563645362854, 0.7677360773086548] 0.7797478795051574 0.03678663755855108
Accuracies: [0.6900826692581177, 0.7272727489471436, 0.7727272510528564, 0.71074378490448, 0.6887966990470886] 0.7179246306419372 0.03087358183932946
Precisions: [0.5961538553237915, 0.6330274939537048, 0.699999988079071, 0.6382978558540344, 0.5746268630027771] 0.6284212112426758 0.04284227212261022
Recals: [0.6526315808296204, 0.7263157963752747, 0.7368420958518982, 0.625, 0.8105263113975525] 0.7102631568908692 0.06573892585773201
